## Import modular

In [1]:
import numpy as np
import tensorflow as tf                    

## Define layers
>add mulitply etal use Tensorflow function

> w is matrix  and b is one index

In [2]:
def add_layer(inputs, in_size,out_size,n_layer,activation_function=None):
    layer_name = 'layer%s'% n_layer
    with tf.name_scope(layer_name): 
        with tf.name_scope('Weights'):
            Weights = tf.Variable(tf.random_normal([in_size,out_size]),name='W')
            tf.summary.histogram(layer_name+'/Weights',Weights)
        with tf.name_scope('biases'):
            biases  = tf.Variable(tf.zeros([1,out_size])+0.1,name='b')
            tf.summary.histogram(layer_name+'/biases',biases)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs,Weights),biases)
    
   
        if activation_function == None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b,)
        tf.summary.histogram(layer_name+'/output',outputs)
    return outputs

## Make up some data

In [3]:
x_data = np.linspace(-1,1,400)[:,np.newaxis]
print(x_data.shape)

(400, 1)


In [4]:
noise = np.random.normal(0,0.05,x_data.shape)

In [5]:
y_data = np.square(x_data) -0.3 + noise

## Set up a model structure

### set placeholder

In [6]:
with tf.name_scope('Inputs'):
    xs = tf.placeholder(tf.float32,[None,1],name='x_input')
    ys = tf.placeholder(tf.float32,[None,1],name='y_input')

### add a hidden layer

In [7]:
l1=add_layer(xs,1,10,n_layer =1,activation_function=tf.nn.relu)

### add a output layer

In [8]:
prediction = add_layer(l1,10,1,2,activation_function=None)

### calculate loss function

In [9]:
with tf.name_scope('Loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(prediction-ys),
                                       reduction_indices=[1]))
    tf.summary.scalar('loss',loss)

### define optimizer

In [10]:
with tf.name_scope('Train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

### define initial operation

In [11]:
init = tf.global_variables_initializer()

## Training

### define a session and collect summary and write file 

In [12]:
sess = tf.Session()
merged = tf.summary.merge_all()

writer = tf.summary.FileWriter('log/',sess.graph)

### step by step training

In [13]:
sess.run(init)

In [14]:
for i in range(1000):
    sess.run(train_step,feed_dict={xs:x_data,ys:y_data})
    if i%20 ==0:
        result = sess.run(merged,feed_dict={xs:x_data,ys:y_data})
        writer.add_summary(result,i)

# Visualize